In [2]:
import numpy as np
#funcion para convertir numero de formato alfanumerico a numerico

def convert_numeric(value):
    value = value.replace('.', '')
    numeric = float(value.split()[1])
    return numeric

PRECIO_DOLAR = 960

In [2]:
import os
import subprocess
import tabula

# Add Java path to PATH environment variable
java_path = r"C:\Program Files\Java\jdk-22\bin"  # Replace with your actual Java path
os.environ['PATH'] = java_path + os.pathsep + os.environ['PATH']

# Verify that Java is now in the PATH
print("Java is in PATH:", subprocess.run(["java", "-version"], capture_output=True, text=True))

Java is in PATH: CompletedProcess(args=['java', '-version'], returncode=0, stdout='', stderr='openjdk version "21.0.6" 2025-01-21\nOpenJDK Runtime Environment JBR-21.0.6+9-895.97-nomod (build 21.0.6+9-b895.97)\nOpenJDK 64-Bit Server VM JBR-21.0.6+9-895.97-nomod (build 21.0.6+9-b895.97, mixed mode, sharing)\n')


# Ejecutar primero

In [3]:
from openpyxl.styles import PatternFill, Font, Alignment

# Define the styles
header_fill = PatternFill(start_color="C5D9F1", end_color="C5D9F1", fill_type="solid")
header_font = Font(bold=True, color="000000", size=12)

title_font = Font(bold=True, color="EEEEEE", size=12)
title_fill = PatternFill(start_color="999999", end_color="999999", fill_type="solid")

mid_title_fill = PatternFill(start_color="EEEEEE", end_color="EEEEEE", fill_type="solid")
mid_title_font = Font(bold=True, color="000000", size=12)

def write_df(ws, df, title, start_row):
    global row_index
    # Write the single DataFrame with its title
    ws.merge_cells(start_row=start_row, start_column=1, end_row=start_row, end_column=len(df.columns))
    title_cell = ws.cell(row=start_row, column=1)
    title_cell.value = title
    title_cell.font = title_font
    title_cell.fill = title_fill
    title_cell.alignment = Alignment(horizontal="center", vertical="center")

    row_index += 1  # Move to the next row

    # Write the DataFrame header
    for col_index, col_name in enumerate(df.columns, start=1):
        cell = ws.cell(row=row_index, column=col_index)
        cell.value = col_name
        cell.font = header_font
        cell.fill = header_fill

    row_index += 1  # Move to the next row

    # Write the DataFrame content
    for r in df.itertuples(index=False):
        for col_index, value in enumerate(r, start=1):
            cell = ws.cell(row=row_index, column=col_index)
            cell.value = value
            # Apply number formatting for columns that should have thousand separators
            # Exclude the "N° de documento" column
            column_header = df.columns[col_index - 1]  # Adjust index since col_index is 1-based
            if isinstance(value, (int, float)) and column_header != 'N° Documento':
                cell.number_format = '#,##0'
        row_index += 1  # Move to the next row

def write_df_dict(ws, df_dict, title, start_row):
    global row_index
    # Write the title for the dictionary
    ws.merge_cells(start_row=start_row, start_column=1, end_row=start_row, end_column=len(df_dict[list(df_dict.keys())[0]].columns))
    title_cell = ws.cell(row=start_row, column=1)
    title_cell.value = title
    title_cell.font = title_font
    title_cell.fill = title_fill
    title_cell.alignment = Alignment(horizontal="center", vertical="center")
    
    row_index = start_row + 1  # Move to the next row

    # Write the shared header
    col_index = 1
    for col_name in df_dict[list(df_dict.keys())[0]].columns:
        cell = ws.cell(row=row_index, column=col_index)
        cell.value = col_name
        cell.font = header_font
        cell.fill = header_fill
        col_index += 1
        
    row_index += 1  # Move to the next row

    # Write each DataFrame in the dictionary, with its title
    for key, df in df_dict.items():
        # Write the key as a title
        ws.merge_cells(start_row=row_index, start_column=1, end_row=row_index, end_column=len(df.columns))
        title_cell = ws.cell(row=row_index, column=1)
        title_cell.value = key
        title_cell.font = mid_title_font
        title_cell.fill = mid_title_fill
        title_cell.alignment = Alignment(horizontal="center", vertical="center")
        
        row_index += 1  # Move to the next row
        
        # Write the DataFrame content
        for r in df.itertuples(index=False):
            for col_index, value in enumerate(r, start=1):
                cell = ws.cell(row=row_index, column=col_index)
                cell.value = value
                
                # Apply number formatting for columns that should have thousand separators
                # Exclude the "N° de documento" column
                column_header = df.columns[col_index - 1]  # Adjust index since col_index is 1-based
                if isinstance(value, (int, float)) and column_header != 'N° Documento' and column_header != 'Puntaje Confianza':
                    cell.value = float(value)
                    cell.number_format = '#,##0'
            row_index += 1  # Move to the next row
        
        row_index += 1  # Add space between DataFrames

## **BCI Cuenta 18**

In [5]:
import pandas as pd
from openpyxl import Workbook
import numpy as np

def formatear_numeros(ws, nombre_columna, start_row):
    """
    Aplica el formato de número con separador de miles a una columna específica.
    """
    col_idx = df.columns.get_loc(nombre_columna) + 1  # +1 porque openpyxl usa índices basados en 1
    for cell in ws.iter_cols(min_col=col_idx, max_col=col_idx, min_row=start_row, max_row=ws.max_row):
        for c in cell:
            c.number_format = '#,##0'  # Formato con separador de miles

# -------------------- ABRIR ARCHIVO --------------------------
#nombre_archivo = "./ERR_03_25/cartolaExcel _bci18_mar25.xlsx"
nombre_archivo = './ERR_05_25/cartolaExcel_may25_bci18.xlsx'
header = 17 # Fila donde están los nombres de la columna
df = pd.read_excel(nombre_archivo, engine='openpyxl', header=header, thousands='.')
#print(df.to_string())
# -------------------- ELIMINA COLUMNAS VACÍAS -----------------------
df = df.loc[:, ~df.columns.str.startswith('Unnamed:')]
# --------------------- SEPARA DATOS -------------------------
dict_df_cta18 = {}
# Separar en DataFrames basados en la columna "Descripción"
df_giro = df[df['Descripción'].str.startswith('GIRO')]
df_transferencia = df[df['Descripción'].str.startswith('TRANSFERENCIA')]
df_cargos = df[~df['Descripción'].str.startswith('GIRO') & ~df['Descripción'].str.startswith('TRANSFERENCIA')]
dict_df_cta18['Cargos'] = df_cargos
dict_df_cta18['Transferencias'] = df_transferencia
dict_df_cta18['Giros'] = df_giro
# -------------------------------------------------------------

# # Esto es para mostrar
# for key in dict_df_cta18:
#     print(dict_df_cta18[key].to_string())

# --- GUARDAR
# Crea un archivo nuevo
wb = Workbook()
# O abre un archivo existente
#workbook_path = 'path_to_your_existing_workbook.xlsx'  # Replace with your file path
#wb = load_workbook(workbook_path)
default_sheet = wb.active
wb.remove(default_sheet)
sheet_name = 'BCIFondRend' # Nombre planilla
ws = wb.create_sheet(title=sheet_name)
row_index = 1
write_df_dict(ws, dict_df_cta18, 'BCI Cuenta 18', row_index)
# Save the workbook
wb.save(f"./out_05_25/bci18.xlsx")

/Users/diegozamora/miniconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [12]:
name_file = './ERR_04_25/cartolaExcel_18_abril25.xlsx'
header = 18 # Fila donde están los nombres de la columna
df = pd.read_excel(name_file, header=header - 1, thousands='.')
df.columns



/Users/diegozamora/miniconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Index(['Fecha', 'Unnamed: 1', 'Sucursal', 'Unnamed: 3', 'Unnamed: 4',
       'Descripción', 'Unnamed: 6', 'N° Documento', 'Unnamed: 8',
       'Cheques y otros cargos', 'Depósitos y Abono', 'Saldo diario'],
      dtype='object')

## **BCI Cuenta 85**

In [4]:
column_dictionary = {'Security' : {'fecha ' : 'Fecha', 'descripción ' : 'Descripción', 'número de documentos ' : 'N° Documento', 'cargos ' : 'Cargo', 'abonos ' : 'Abono', 'saldos ' : 'Saldo'},
                     'Siteminder' : {'Referencia de la reserva' : 'ID Reserva', 'Nombres de los huéspedes' : 'Huéspedes', 'Llegada' : 'Check-In', 'Salida' : 'Check-Out', 'Affiliated Channel' :'Canal afiliado', 'Estado de la reserva' : 'Estado', 'Ocupación' : 'Noches', 'Habitación' : 'Habitaciones', 'Precio total' : 'Total'},
                     'Transbank' : {'Fecha' : 'Fecha Venta', 'Local' : 'Codigo comercio', 'Tipo Movimiento' : 'Tipo movimiento', 'Identificador' : 'Número tarjeta', 'Monto Afecto' : 'Total', 'N° Boleta' : 'Número'},
                     'Transferencias' : {'Monto $' : 'CARGO TRANSF', 'Nombre Destino/Origen' : 'NOMBRE DESTINO', 'Mensaje Destino' : 'MENSAJE'},
                     'Cuenta BCI Comercio Exterior' : {'SUCURSAL' : 'OFICINA', 'CHEQUES Y OTROS CARGOS' : 'CARGO', 'DEPOSITOS Y ABONOS' : 'ABONO', 'SALDO DIARIO' : 'SALDO'},
                     'Cuenta BCI 85' : {'Fecha' : 'FECHA', 
                                        'Sucursal' : 'OFICINA', 
                                        'Descripción' : 'MOVIMIENTO', 
                                        'N° Documento': 'N° DOCUMENTO', 
                                        'Cheques y otros cargos' : 'CARGO',
                                        'Depósitos y Abono' : 'ABONO',
                                        'Saldo diario' : 'SALDO'},
                     'Cuenta BCI 18' : {'Fecha' : 'FECHA', 
                                        'Sucursal' : 'OFICINA', 
                                        'Descripción' : 'MOVIMIENTO', 
                                        'N° Documento': 'N° DOCUMENTO', 
                                        'Cheques y otros cargos' : 'CARGO',
                                        'Depósitos y Abono' : 'ABONO',
                                        'Saldo diario' : 'SALDO'},
                     'Tarjeta de crédito nacional 24' : {'FECHA OPERACIÓN' : 'FECHA', 
                                                         'CODIGO REFERENCIA' : 'CÓDIGO REFERENCIA', 
                                                         'LUGAR DE OPERACIÓN' : 'LUGAR DE OPERACIÓN', 
                                                         'DESCRIPCION OPERACION O COBRO': 'DESCRIPCION', 
                                                         'MONTO OPERACIÓN O COBRO' : 'CARGO'},
                     'Tarjeta de crédito nacional 69' : {'FECHA OPERACIÓN' : 'FECHA', 
                                                         'CODIGO REFERENCIA' : 'CÓDIGO REFERENCIA', 
                                                         'LUGAR DE OPERACIÓN' : 'LUGAR DE OPERACIÓN', 
                                                         'DESCRIPCION OPERACION O COBRO': 'DESCRIPCION', 
                                                         'MONTO OPERACIÓN O COBRO' : 'CARGO'},
                     'Tarjeta de crédito internacional' : {'FECHA OPERACIÓN' : 'FECHA', 
                                                           'NUMERO REFERENCIA INTERNACIONAL' : 'CÓDIGO REFERENCIA', 
                                                           'DESCRIPCION OPERACION O COBRO' : 'DESCRIPCION'}}

In [5]:
from openpyxl import load_workbook
from text_classification import classify_text
import pickle
from openpyxl import Workbook
import pandas as pd

# -------------------- ABRIR ARCHIVO CARTOLA --------------------------
#nombre_archivo = './ERR_03_25/cartolaExcel_bc85_mar25.xlsx'
nombre_archivo = './ERR_05_25/cartolaExcel_may25_bci85.xlsx'
header = 22 # Fila donde están los nombres de la columna (Numero de fila en excel - 1)
engine = 'openpyxl'
df_cartola = pd.read_excel(nombre_archivo, header=header, thousands='.')
# --------------- ABRIR ARCHIVO TRANSFERENCIAS ------------------------
#nombre_archivo = './ERR_03_25/cartola_04042025145109_trasf_mar25.xls'
nombre_archivo = './ERR_05_25/cartola_20062025094812_may25_tranf_bci.xls'
engine = 'xlrd' # El motor openpyxl no usa xls
header = 8 # Fila donde están los nombres de la columna (Numero de fila en excel - 1)
df_transfer = pd.read_excel(nombre_archivo, header=header, thousands='.')
# ----------------------------------------------------------------------

df_cartola.rename(columns=column_dictionary['Cuenta BCI 85'], inplace=True)
df_transfer.rename(columns=column_dictionary['Transferencias'], inplace=True)

df_transfer = df_transfer.copy()
df_cartola = df_cartola.copy()

df_transfer['CARGO TRANSF'] = df_transfer['CARGO TRANSF'].apply(convert_numeric)

# df_cartola['Fecha'] = pd.to_datetime(df_cartola['Fecha'])
# df_transfer['Fecha'] = pd.to_datetime(df_transfer['Fecha'])

# Step 1: Filter df_cartola for rows where the description is in desc_list or contains the word 'Transfer'
desc_list = ['TRASPASO FONDOS OTRO BANCO EN LINEA',  'CARGO POR TRANSF DE FONDOS AUTOSERVICIO']
filtered_cartola = df_cartola[
    df_cartola['MOVIMIENTO'].isin(desc_list) |  # Descriptions from desc_list
    df_cartola['MOVIMIENTO'].str.contains('Transfer', case=False, na=False)  # Descriptions containing 'Transfer'
]

# Create an identifier for rows in df_cartola where the description matches
filtered_cartola['merge_id'] = filtered_cartola.groupby(['CARGO']).cumcount()
# Create a similar identifier in df_transfer, considering the rows with 'aguinaldo'
df_transfer['merge_id'] = df_transfer.groupby(['CARGO TRANSF']).cumcount()

# Step 2: Perform the merge
merged_filtered_cartola = filtered_cartola.merge(
    df_transfer[['CARGO TRANSF', 'NOMBRE DESTINO', 'MENSAJE', 'merge_id']],
    how='left',
    left_on=['CARGO', 'merge_id'],  # Column in df_cartola
    right_on=['CARGO TRANSF', 'merge_id']  # Column in df_transfer
    )

# # Step 2: Perform the merge
# # La fecha no coincide
# merged_filtered_cartola = filtered_cartola.merge(
#     df_transfer[['Monto $', 'Nombre Destino/Origen', 'Mensaje Destino']],
#     how='left',
#     left_on=['Cheques y otros cargos'],  # Column in df_cartola
#     right_on=['Monto $']  # Column in df_transfer
#     )

# Step 3: Get the rest of the df_cartola that wasn't filtered
non_filtered_cartola = df_cartola[
    ~df_cartola['MOVIMIENTO'].isin(desc_list) &  # Exclude rows matching desc_list
    ~df_cartola['MOVIMIENTO'].str.contains('Transfer', case=False, na=False)  # Exclude rows containing 'Transfer'
]
# Step 4: Combine the merged result with the non-filtered rows
final_df = pd.concat([merged_filtered_cartola, non_filtered_cartola])
# Step 5: Optional - Sort the final DataFrame to retain the original order if needed
final_df = final_df.sort_index()
# --------------------------- AJUSTES -----------------------------------------
final_df = final_df.drop(columns=['CARGO TRANSF'])
final_df = final_df.loc[:, ~final_df.columns.str.startswith('Unnamed:')]
# --------------------------------------------------------------------
df_cta85 = final_df
df_cta85['CLAVE'] = ''

# APLICACION MODELO PREDICTIVO ---
# Cargar modelo
with open("./modelo_C85.pkl", "rb") as model_file, open("vectorizer_C85.pkl", "rb") as vectorizer_file:
    model = pickle.load(model_file)  # Load the model
    vectorizer = pickle.load(vectorizer_file)  # Load the vectorizer
# Aplicar
df_cta85[['CLAVE', 'PREDICCIÓN']] = df_cta85.apply(
    lambda row: pd.Series(classify_text(model, vectorizer, row['MOVIMIENTO'], row['NOMBRE DESTINO'], row['MENSAJE'])),
    axis=1
)
df_cta85 = df_cta85[['FECHA', 'OFICINA', 'MOVIMIENTO', 'N° DOCUMENTO', 'CARGO', 'ABONO', 'SALDO', 'CLAVE', 'PREDICCIÓN', 'NOMBRE DESTINO', 'MENSAJE']]
print(df_cta85.to_string())

# # --- GUARDAR
# # Crea un archivo nuevo
# wb = Workbook()
# # O abre un archivo existente
# #workbook_path = 'path_to_your_existing_workbook.xlsx'  # Replace with your file path
# #wb = load_workbook(workbook_path)
# default_sheet = wb.active
# wb.remove(default_sheet)
# sheet_name = 'BCI' # Nombre planilla
# ws = wb.create_sheet(title=sheet_name)
# row_index = 1
# write_df(ws, df_cta85, 'BCI Cuenta 85', row_index)
# # Save the workbook
# wb.save(f"./out_05_25/bci85.xlsx")

         FECHA    OFICINA                               MOVIMIENTO  N° DOCUMENTO      CARGO       ABONO     SALDO                   CLAVE  PREDICCIÓN                                 NOMBRE DESTINO                         MENSAJE
0   02/05/2025  OF CENTRA  CARGO POR TRANSF DE FONDOS AUTOSERVICIO  9.540109e+10  5000000.0         NaN  38352834         Fondos a Rendir         0.2  Soc Hotelera Zamora Ramirez hnos Ltda          transferencia  pagar impuestos
1   02/05/2025  OF CENTRA      TRASPASO FONDOS OTRO BANCO EN LINEA  1.174333e+09   130000.0         NaN  38222834         Costos Directos         0.3  Ernesto Perez Flores                            Agrovet fact 48 hotel Pascual
2   05/05/2025  OF CENTRA  CARGO POR TRANSF DE FONDOS AUTOSERVICIO  9.765431e+08    82224.0         NaN  38140610         Costos Directos         0.3  QUALITY WATER SERVICE CHILE SPA                     factura 436359 soc Zamora
3   05/05/2025      MACUL        PAGO CREDITO D05500025718 049/060  5.500026e+09   6

In [8]:
from openpyxl import load_workbook
from openpyxl.utils import column_index_from_string
import pandas as pd
import copy

from openpyxl.utils import column_index_from_string
from openpyxl.worksheet.datavalidation import DataValidation
from openpyxl.utils import get_column_letter

def write_df_to_named_table_by_header(ws, sheet_name, table_name, df):
    """
    Overwrites the contents of an Excel table with data from a pandas DataFrame,
    matching on column names and expanding the table range if needed.
    Also re-applies data validation from header row to new rows.
    """

    table = ws.tables.get(table_name)
    if table is None:
        raise ValueError(f"Table '{table_name}' not found in sheet '{sheet_name}'.")

    # Parse range
    start_cell, _ = table.ref.split(':')
    start_col_letter = ''.join(filter(str.isalpha, start_cell))
    start_col = column_index_from_string(start_col_letter)
    start_row = int(''.join(filter(str.isdigit, start_cell)))

    # Read headers
    headers = [ws.cell(row=start_row, column=col_idx).value for col_idx in range(start_col, start_col + len(table.tableColumns))]
    header_to_col = {header: col_idx for header, col_idx in zip(headers, range(start_col, start_col + len(headers)))}

    # Write data
    for row_offset, (_, df_row) in enumerate(df.iterrows(), start=1):
        for header, col_idx in header_to_col.items():
            if header in df.columns:
                ws.cell(row=start_row + row_offset, column=col_idx, value=df_row[header])

    # Expand table ref
    new_end_row = start_row + len(df)
    new_ref = f"{start_cell}:{ws.cell(row=new_end_row, column=start_col + len(headers) - 1).coordinate}"
    table.ref = new_ref

    # 🟡 Re-apply Data Validation (if exists in header row)
    for header, col_idx in header_to_col.items():
        col_letter = get_column_letter(col_idx)
        source_cell = ws[f"{col_letter}{start_row + 1}"]  # Row +1 is first data row, usually has DV
        for dv in ws.data_validations.dataValidation:
            if source_cell.coordinate in dv.cells:
                new_range = f"{col_letter}{start_row + 1}:{col_letter}{new_end_row}"
                new_dv = DataValidation(
                    type=dv.type,
                    formula1=dv.formula1,
                    allow_blank=dv.allow_blank,
                    showDropDown=dv.showDropDown,
                    showErrorMessage=dv.showErrorMessage,
                    error=dv.error,
                    errorTitle=dv.errorTitle,
                    prompt=dv.prompt,
                    promptTitle=dv.promptTitle
                )
                new_dv.add(new_range)
                ws.add_data_validation(new_dv)
                break  # Don't duplicate

    return new_end_row

# def write_df_to_named_table_by_header(ws, sheet_name, table_name, df):
#     """
#     Overwrites the contents of an Excel table with data from a pandas DataFrame,
#     matching on column names and expanding the table range if needed.
#     """

#     # ✅ Get the Table object correctly
#     table = ws.tables.get(table_name)
#     if table is None:
#         raise ValueError(f"Table '{table_name}' not found in sheet '{sheet_name}'.")

#     # Parse the range of the table (e.g. 'B2:F10')
#     start_cell, end_cell = table.ref.split(':')
#     start_col_letter = ''.join(filter(str.isalpha, start_cell))
#     start_col = column_index_from_string(start_col_letter)
#     start_row = int(''.join(filter(str.isdigit, start_cell)))

#     # Get headers
#     headers = [ws.cell(row=start_row, column=col_idx).value for col_idx in range(start_col, start_col + len(table.tableColumns))]
#     header_to_col = {header: col_idx for header, col_idx in zip(headers, range(start_col, start_col + len(headers)))}

#     # Write data to matching columns
#     for row_offset, (_, df_row) in enumerate(df.iterrows(), start=1):
#         for header, col_idx in header_to_col.items():
#             if header in df.columns:
#                 ws.cell(row=start_row + row_offset, column=col_idx, value=df_row[header])

#     # Expand table range
#     new_end_row = start_row + len(df)
#     new_ref = f"{start_cell}:{ws.cell(row=new_end_row, column=start_col + len(headers) - 1).coordinate}"
#     table.ref = new_ref

#     end_row = new_end_row
#     return end_row
#     # wb.save(file_path)
#     # wb.close()

from openpyxl.utils import column_index_from_string
import copy  # Required to deep copy styles
from openpyxl.styles import Font, PatternFill, Border, Alignment, Protection

def move_table_down(ws, table, offset_rows):
    start_cell, end_cell = table.ref.split(':')
    start_col_letter = ''.join(filter(str.isalpha, start_cell))
    start_row = int(''.join(filter(str.isdigit, start_cell)))

    end_col_letter = ''.join(filter(str.isalpha, end_cell))
    end_row = int(''.join(filter(str.isdigit, end_cell)))

    start_col = column_index_from_string(start_col_letter)
    end_col = column_index_from_string(end_col_letter)

    # Copy cells downwards from bottom to top
    for row in range(end_row, start_row - 1, -1):
        for col in range(start_col, end_col + 1):
            source = ws.cell(row=row, column=col)
            target = ws.cell(row=row + offset_rows, column=col)

            target.value = source.value

            if source.has_style:
                target.font = copy.copy(source.font)
                target.fill = copy.copy(source.fill)
                target.border = copy.copy(source.border)
                target.alignment = copy.copy(source.alignment)
                target.number_format = source.number_format  # This is a string
                target.protection = copy.copy(source.protection)

            source.value = None
            source.font = Font()
            source.fill = PatternFill()
            source.border = Border()
            source.alignment = Alignment()
            source.number_format = 'General'
            source.protection = Protection()

    # Update table ref
    new_start_row = start_row + offset_rows
    new_end_row = end_row + offset_rows
    table.ref = f"{start_col_letter}{new_start_row}:{end_col_letter}{new_end_row}"

from openpyxl import load_workbook

from openpyxl.utils import range_boundaries, get_column_letter
def move_table_and_label_down(ws, table, offset, label_rows_above=1):
    """
    Move a table and its label (above it) down by `offset` rows.
    
    Parameters:
    - ws: Worksheet object
    - table: openpyxl.table.Table object
    - offset: how many rows to move
    - label_rows_above: how many rows above the table are considered part of the label
    """
    # Parse table bounds
    min_col, min_row, max_col, max_row = range_boundaries(table.ref)
    
    # Compute full range including label
    label_start_row = max(min_row - label_rows_above, 1)
    start_cell = f"{get_column_letter(min_col)}{label_start_row}"
    end_cell = f"{get_column_letter(max_col)}{max_row}"
    full_range = f"{start_cell}:{end_cell}"

    # Move the entire range (label + table)
    ws.move_range(full_range, rows=offset, cols=0)

    # Update table reference to new position
    new_min_row = min_row + offset
    new_max_row = max_row + offset
    new_start = f"{get_column_letter(min_col)}{new_min_row}"
    new_end = f"{get_column_letter(max_col)}{new_max_row}"
    table.ref = f"{new_start}:{new_end}"


In [9]:
# STEP 1 — load workbook and sheet
file_path = './Book 1.xlsx'
sheet_name = 'BCI'
wb = load_workbook(file_path)
ws = wb[sheet_name]

# STEP 2 — get tables and data
table1 = ws.tables.get('Cuenta_85')
table2 = ws.tables.get('BCI_Comercio_Exterior')
df1 = df_cta85
df2 = pd.read_csv('cuenta_comercio_ext.csv')

# STEP 3 — move second table first
# (assume both tables already exist and table1 is visually above table2)
start_cell_1, _ = table1.ref.split(':')
start_row_1 = int(''.join(filter(str.isdigit, start_cell_1)))

start_cell_2, _ = table2.ref.split(':')
start_row_2 = int(''.join(filter(str.isdigit, start_cell_2)))

offset = start_row_1 + len(df1) - start_row_2 + 3  # +1 for spacing
if offset > 0:
    #move_table_down(ws, table2, offset)
    move_table_and_label_down(ws, table2, offset, label_rows_above=1)

# STEP 4 — now write data safely
last_row_1 = write_df_to_named_table_by_header(ws, sheet_name, 'Cuenta_85', df1)
last_row_2 = write_df_to_named_table_by_header(ws, sheet_name, 'BCI_Comercio_Exterior', df2)

# STEP 5 — save
wb.save('your_filled_file.xlsx')
wb.close()

In [ ]:
from openpyxl import load_workbook
from text_classification import classify_text
import pickle
from openpyxl import Workbook
import pandas as pd

nombre_archivo = './ERR_04_25/cartolaExcel_85_abril25.xlsx'
header = 23
df_cartola = pd.read_excel(nombre_archivo, engine='openpyxl', header=header - 1, thousands='.')
# Transferencias
nombre_archivo = './ERR_04_25/cartola_10052025100828_tranf_abril25.xls'
engine = 'xlrd' # El motor openpyxl no usa xls
header = 9 
df_transfer = pd.read_excel(nombre_archivo, engine=engine, header=header - 1, thousands='.')

df_transfer['Monto $'] = df_transfer['Monto $'].apply(convert_numeric)

# Step 1: Filter df_cartola for rows where the description is in desc_list or contains the word 'Transfer'
desc_list = ['TRASPASO FONDOS OTRO BANCO EN LINEA',  'CARGO POR TRANSF DE FONDOS AUTOSERVICIO']
filtered_cartola = df_cartola[
    df_cartola['Descripción'].isin(desc_list) |  # Descriptions from desc_list
    df_cartola['Descripción'].str.contains('Transfer', case=False, na=False)  # Descriptions containing 'Transfer'
]

# Create an identifier for rows in df_cartola where the description matches
filtered_cartola['merge_id'] = filtered_cartola.groupby(['Cheques y otros cargos']).cumcount()
# Create a similar identifier in df_transfer, considering the rows with 'aguinaldo'
df_transfer['merge_id'] = df_transfer.groupby(['Monto $']).cumcount()

# Step 2: Perform the merge
merged_filtered_cartola = filtered_cartola.merge(
    df_transfer[['Monto $', 'Nombre Destino/Origen', 'Mensaje Destino', 'merge_id']],
    how='left',
    left_on=['Cheques y otros cargos', 'merge_id'],  # Column in df_cartola
    right_on=['Monto $', 'merge_id']  # Column in df_transfer
    )

# Step 3: Get the rest of the df_cartola that wasn't filtered
non_filtered_cartola = df_cartola[
    ~df_cartola['Descripción'].isin(desc_list) &  # Exclude rows matching desc_list
    ~df_cartola['Descripción'].str.contains('Transfer', case=False, na=False)  # Exclude rows containing 'Transfer'
]
# Step 4: Combine the merged result with the non-filtered rows
df = pd.concat([merged_filtered_cartola, non_filtered_cartola])
# Step 5: Optional - Sort the final DataFrame to retain the original order if needed
df = df.sort_index()

# APLICACION MODELO PREDICTIVO ---
# Cargar modelo
with open("./modelo_C85.pkl", "rb") as model_file, open("vectorizer_C85.pkl", "rb") as vectorizer_file:
    model = pickle.load(model_file)  # Load the model
    vectorizer = pickle.load(vectorizer_file)  # Load the vectorizer
# Aplicar
df[['Predicción Clave', 'Puntaje Confianza']] = df.apply(
    lambda row: pd.Series(classify_text(model, vectorizer, row['Descripción'], row['Nombre Destino/Origen'], row['Mensaje Destino'])),
    axis=1
)

print(df.to_string())

         Fecha  Unnamed: 1   Sucursal  Unnamed: 3  Unnamed: 4                              Descripción  Unnamed: 6  N° Documento  Unnamed: 8  Cheques y otros cargos  Depósitos y Abono  Saldo diario  merge_id    Monto $                          Nombre Destino/Origen                 Mensaje Destino        Predicción Clave  Puntaje Confianza
0   01/04/2025         NaN  OF CENTRA         NaN         NaN  CARGO POR TRANSF DE FONDOS AUTOSERVICIO         NaN  9.540109e+08         NaN               4500000.0                NaN      28082505       0.0  4500000.0  SOCIEDAD HOTELERA ZAMORA RAMIREZ HERMANOS LIM                                         Fondos a Rendir                0.8
1   02/04/2025         NaN   UGCA AUT         NaN         NaN                  STARKEN 1689 PENA 39290         NaN  3.929000e+04         NaN                 48040.0                NaN      28034465       NaN        NaN                                            NaN                             NaN        Gastos Operac

## **BCI Comercio Exterior**

In [2]:
#se importa un pdf, antes se debe grabar un psf desde la página del banco (imporimir) previamente se debe sacar la paleta de la izquierda   



import pdfplumber
from collections import defaultdict
from text_classification import classify_text
import importlib
import openpyxl
importlib.reload(openpyxl)
import pandas as pd
import pickle
pdf_path = './ERR_03_25/bel.bci.cl_cl_bci_aplicaciones_contenidoLayoutOSSPyme.jsf_mar25.pdf'
import tabula.io as tabula

tables = tabula.read_pdf(pdf_path, pages='all', multiple_tables=True, encoding='utf-8')
dict_col = {}
print(tables)
df_cta_cte = tables[0]

# Initialize indices
start_index = None
end_index = None

#print(df_cta_cte.to_string())

# Iterate through the rows to find the indices of "FECHA" and "Resumen"
for i, row in df_cta_cte.iterrows():
    if str(row[0]).startswith('FECHA'):
        start_index = i
    if str(row[0]).startswith('Resumen'):
        end_index = i
        break  # Stop iteration once "Resumen" is found
df_cta_cte = df_cta_cte.iloc[start_index + 1 : end_index]

# ACA PUEDEN HABER PROBLEMAS
df_cta_cte['Fecha'] = df_cta_cte.iloc[:, 0].apply(lambda x: x.split()[0])
df_cta_cte['Sucursal'] = df_cta_cte.iloc[:, 0].apply(lambda x: ' '.join(x.split()[1:]))

# Drop the original first column if no longer needed
df_cta_cte = df_cta_cte.drop(df_cta_cte.columns[0], axis=1)
columns_order = ['Fecha', 'Sucursal'] + df_cta_cte.columns[:-2].tolist()  # Move the new columns to the front
df_cta_cte = df_cta_cte[columns_order]
new_column_names = ['Fecha', 'Sucursal', 'Descripción', 'N° Documento', 'Cargo', 'Abono', 'Saldo']
df_cta_cte.columns = new_column_names

def format_number(row):
    if type(row) == str:
        return float(row.replace('.', '').replace(',', '.'))
for col in ['Cargo', 'Abono', 'Saldo']:
    df_cta_cte[col] = df_cta_cte[col].apply(format_number)


# APLICACION MODELO PREDICTIVO ---
# Cargar modelo
with open("./modelo_CUSD.pkl", "rb") as model_file, open("vectorizer_CUSD.pkl", "rb") as vectorizer_file:
    model = pickle.load(model_file)  # Load the model
    vectorizer = pickle.load(vectorizer_file)  # Load the vectorizer
# Aplicar
df_cta_cte[['Predicción Clave', 'Puntaje Confianza']] = df_cta_cte.apply(
    lambda row: pd.Series(classify_text(model, vectorizer, row['Descripción'])),
    axis=1
)

print(df_cta_cte.to_string())

# --- GUARDAR
# Crea un archivo nuevo
wb = openpyxl.Workbook()
# O abre un archivo existente
#workbook_path = 'path_to_your_existing_workbook.xlsx'  # Replace with your file path
#wb = load_workbook(workbook_path)
default_sheet = wb.active
wb.remove(default_sheet)
sheet_name = 'BCI' # Nombre planilla
ws = wb.create_sheet(title=sheet_name)
row_index = 1
write_df(ws, df_cta_cte, 'BCI Cuenta Comercio Exterior', row_index)
# Save the workbook
wb.save(f"./out_03_25/bciext.xlsx")

[             Búsqueda rápida Unnamed: 0  \
0                         NaN        NaN   
1                         NaN        NaN   
2                         NaN        NaN   
3                         NaN        NaN   
4                         NaN        NaN   
5                         NaN        NaN   
6                         NaN        NaN   
7                         NaN        NaN   
8                         NaN        NaN   
9                         NaN        NaN   
10                        NaN        NaN   
11                        NaN        NaN   
12                        NaN        NaN   
13                        NaN        NaN   
14                        NaN        NaN   
15                        NaN        NaN   
16                        NaN        NaN   
17                        NaN        NaN   
18                        NaN        NaN   
19                        NaN        NaN   
20                        NaN        NaN   
21                        NaN  

AttributeError: 'float' object has no attribute 'split'

## **TC NACIONAL 24**

In [ ]:
import pdfplumber
from collections import defaultdict
import pandas as pd
pdf_path = './documentos_0125/.pdf'

tables = []
with pdfplumber.open(pdf_path) as pdf:
    page_list = pdf.pages
    for page_number, page in enumerate(page_list):
        if page_number == 0:
            # En la primera página de índice 0, la tabla que nos interesa siempre es la última
            # accedemos a ella con el índice -1
            # Además, resulta conveniente leer la tabla desde la tercera columna, desde aquí empieza
            # por ejemplo: '1.TOTAL OPERACIONES'. Entonces usamos índice [3:] para leer desde la tercera en adelante.
            table = page.extract_tables()[-1][3:] 
        elif page_number > 0:
            # Hasta ahora los documentos tienen solo dos páginas, pero en el caso de que se alargaran
            # siempre la tabla de interés es la primera y entonces usamos el índice 0.
            # Nos saltamos las tres primeras filas de la tabla.
            table = page.extract_tables()[0][3:]
        tables.append(table)
        
# La tabla se guarda como una lista de listas. Cada sublista es una columna completa y los elementos de las columnas se guardan 
# como strings separados con el caractér \n

# En esta linea debemos listar los nombres de las secciones de la tabla.
header_list = ['1.TOTAL OPERACIONES', '2.PRODUCTOS O SERVICIOS VOLUNTARIAMENTE CONTRATADOS SIN MOVIMIENTOS', '3.CARGOS, COMISIONES, IMPUESTOS Y ABONOS']
# Aquí van los nombres que le vamos a dar a las columnas que queremos extraer.
col_list = ['Lugar', 'Fecha', 'Código referencia', 'Descripción', 'Monto']

dict_head = {}
current_header = None
for table in tables:
    for row in table:
        # No nos interesan las tres últimas columnas
        row = row[:-3]
        if row[0].startswith(tuple(header_list)): # Si el primer elemento en la lista 'table' es uno de los headers
            current_header = row[0].split(' (')[0] # Actualiza la variable current_header
            dict_head[current_header] = defaultdict(list) # El elemento del diccionario que tiene como KEY el header actual, por ejemplo> 1. TOTAL OPERACIONES
                                                          # es otro diccionario cuyos KEYS son el nombre de las columnas en col_list
        else: # Si el primer elemento de la lista 'table' no es un header, entonces es contenido de la tabla.
            for i, element in enumerate(row): # Cada elemento en row es una columna completa
                dict_head[current_header][col_list[i]] += element.split('\n') # Se separan los elementos de la columna con el split('\n')

# Esta es la parte más manual. Hay una fila en la columna DESCRIPCIÓN de la tabla '1. TOTAL OPERACIONES' que no queremos incluir
# dice TOTAL TARJETA XXXXXXXXXXXX8724
# Tampoco incluimos lo que está arriba de la fila TOTAL PAGOS A LA CUENTA
# Buscamos los índices de esas filas para después omitirlas
index_mid_head = None
index_bott_head = None
for i, row in enumerate(dict_head['1.TOTAL OPERACIONES']['Descripción']):
    if row == 'TOTAL PAGOS A LA CUENTA':
        index_mid_head = i
    elif row == 'TOTAL TARJETA XXXXXXXXXXXX8724':
        index_bott_head = i

# Para llevar esto al Excel primero lo pasamos a un DataFrame. En este bloque se guardan los DataFrames
# correspondientes a cada subtabla en un diccionario.
# Debemos corregir algunos detalles, entre estos el de la fila TOTAL TARJETA XXXXXXXXXXXX8724
dict_df_nacional24 = {}
for header in header_list:
    df = pd.DataFrame()
    for i, col in enumerate(col_list):
        if header == '1.TOTAL OPERACIONES':
            if col == 'Lugar':
                df[col] = dict_head[header][col]
            elif col == 'Descripción':
                df[col] = dict_head[header][col][index_mid_head + 1 : index_bott_head] # Leemos entre TOTAL PAGOS A LA CUENTA y TOTAL TARJETA ...
            else:
                df[col] = dict_head[header][col][index_mid_head : ] # Leemos desde TOTAL PAGOS A LA CUENTA
        elif header == '3.CARGOS, COMISIONES, IMPUESTOS Y ABONOS':
            if col == 'Lugar':
                df[col] = dict_head[header][col] * len(dict_head[header]['Monto'])
            else:
                df[col] = dict_head[header][col]
        else:
            df[col] = dict_head[header][col]
    dict_df_nacional24[header] = df

# Eliminamos el símbolo $ y el punto separador de miles
for key in dict_df_nacional24:
    dict_df_nacional24[key]['Monto'] = dict_df_nacional24[key]['Monto'].apply(lambda x: int(x.split()[1].replace('.','')))

# APLICACION MODELO PREDICTIVO ---
# Cargar modelo
with open("./modelo_TCN.pkl", "rb") as model_file, open("vectorizer_TCN.pkl", "rb") as vectorizer_file:
    model = pickle.load(model_file)  # Load the model
    vectorizer = pickle.load(vectorizer_file)  # Load the vectorizer
for key in dict_df_nacional24:
    df_key = dict_df_nacional24[key]
    if df_key.empty:
        print(f"The DataFrame for key '{key}' is empty. Skipping...")
        continue
    df_key[['Predicción Clave', 'Puntaje Confianza']] = df_key.apply(
        lambda row: pd.Series(classify_text(model, vectorizer, row['Descripción']), index=['Predicción Clave', 'Puntaje Confianza']), 
        axis=1
    )


# Esto es para mostrar
from IPython.display import display
for key in dict_df_nacional24:
    display(dict_df_nacional24[key])


# --- GUARDAR
# Crea un archivo nuevo
wb = Workbook()
# O abre un archivo existente
#workbook_path = 'path_to_your_existing_workbook.xlsx'  # Replace with your file path
#wb = load_workbook(workbook_path)
default_sheet = wb.active
wb.remove(default_sheet)
sheet_name = 'TC Nacional 24' # Nombre planilla
ws = wb.create_sheet(title=sheet_name)
row_index = 1
write_df_dict(ws, dict_df_nacional24, 'TC Nacional 24', row_index)
# Save the workbook
wb.save(f"tc24.xlsx")

ValueError: Length of values (3) does not match length of index (2)

## **TC NACIONAL 69**

In [7]:
import pdfplumber
from collections import defaultdict
import pandas as pd
pdf_path = r'C:\Users\czamora\Downloads\EstadoDeCuentaNacional_69.pdf'
pdf_path = './documentos/EstadoDeCuentaNacional_8069.pdf'
pdf_path = './documentos_1224/EstadoDeCuentaNacional_69_dic24.pdf'
pdf_path = './documentos_0125/EstadoDeCuentaNacional_69_ene25.pdf'

tables = []
with pdfplumber.open(pdf_path) as pdf:
    page_list = pdf.pages
    for page_number, page in enumerate(page_list):
        if page_number == 0:
            # En la primera página de índice 0, la tabla que nos interesa siempre es la última
            # accedemos a ella con el índice -1
            # Además, resulta conveniente leer la tabla desde la tercera columna, desde aquí empieza
            # por ejemplo: '1.TOTAL OPERACIONES'. Entonces usamos índice [3:] para leer desde la tercera en adelante.
            table = page.extract_tables()[-1][3:] 
        elif page_number > 0:
            # Hasta ahora los documentos tienen solo dos páginas, pero en el caso de que se alargaran
            # siempre la tabla de interés es la primera y entonces usamos el índice 0.
            # Nos saltamos las tres primeras filas de la tabla.
            table = page.extract_tables()[0][3:]
        tables.append(table)

# La tabla se guarda como una lista de listas. Cada sublista es una columna completa y los elementos de las columnas se guardan 
# como strings separados con el caractér \n

# En esta linea debemos listar los nombres de las secciones de la tabla.
header_list = ['1.TOTAL OPERACIONES', '2.PRODUCTOS O SERVICIOS VOLUNTARIAMENTE CONTRATADOS SIN MOVIMIENTOS', '3.CARGOS, COMISIONES, IMPUESTOS Y ABONOS']
# Aquí van los nombres que le vamos a dar a las columnas que queremos extraer.
col_list = ['Lugar', 'Fecha', 'Código referencia', 'Descripción', 'Monto']

dict_head = {}
current_header = None
for table in tables:
    for row in table:
        # No nos interesan las tres últimas columnas
        row = row[:-3]
        if row[0].startswith(tuple(header_list)): # Si el primer elemento en la lista 'table' es uno de los headers
            current_header = row[0].split(' (')[0] # Actualiza la variable current_header
            dict_head[current_header] = defaultdict(list) # El elemento del diccionario que tiene como KEY el header actual, por ejemplo> 1. TOTAL OPERACIONES
                                                          # es otro diccionario cuyos KEYS son el nombre de las columnas en col_list
        else: # Si el primer elemento de la lista 'table' no es un header, entonces es contenido de la tabla.
            for i, element in enumerate(row): # Cada elemento en row es una columna completa
                dict_head[current_header][col_list[i]] += element.split('\n') # Se separan los elementos de la columna con el split('\n')

dict_df_nacional69 = {}
col_len = None
string = 'TOTAL TARJETA XXXXXXXXXXXX8069'
for header in header_list:
    print(header)
    df = pd.DataFrame()
    for i, col in enumerate(col_list):
        if header == '1.TOTAL OPERACIONES':
            if string in dict_head[header][col]:
                dict_head[header][col].remove(string)
            if i == 0:
                col_len = len(dict_head[header][col])
            df[col] = dict_head[header][col][-12:]
        elif header == '3.CARGOS, COMISIONES, IMPUESTOS Y ABONOS':
            if col == 'Lugar':
                df[col] = dict_head[header][col] * len(dict_head[header]['Monto'])
            else:
                df[col] = dict_head[header][col]
        else:
            df[col] = dict_head[header][col]
    dict_df_nacional69[header] = df

for key in dict_df_nacional69:
    print(dict_df_nacional69[key].to_string())

def remove_dollar(string):
    split_list = string.split()
    if '$' in split_list:
        split_list.remove('$')
        out_string = ' '.join(split_list)
        return out_string
    else:
        return string
def string_int(string):
    return int(string.replace('.',''))
    

# Por alguna razón, en este PDF se lee el símbolo $ en la columna Descripción y no junto con los números en la columna Monto
# Eliminamos el punto separador de miles y el $
for key in dict_df_nacional69:
    if key == '1.TOTAL OPERACIONES':
        dict_df_nacional69[key]['Descripción'] = dict_df_nacional69[key]['Descripción'].apply(remove_dollar)
        dict_df_nacional69[key]['Monto'] = dict_df_nacional69[key]['Monto'].apply(remove_dollar)
        dict_df_nacional69[key]['Monto'] = dict_df_nacional69[key]['Monto'].apply(string_int)
    elif key == '3.CARGOS, COMISIONES, IMPUESTOS Y ABONOS':
        dict_df_nacional69[key]['Monto'] = dict_df_nacional69[key]['Monto'].apply(remove_dollar)
        dict_df_nacional69[key]['Monto'] = dict_df_nacional69[key]['Monto'].apply(string_int)

# APLICACION MODELO PREDICTIVO ---
# Cargar modelo
with open("./modelo_TCN.pkl", "rb") as model_file, open("vectorizer_TCN.pkl", "rb") as vectorizer_file:
    model = pickle.load(model_file)  # Load the model
    vectorizer = pickle.load(vectorizer_file)  # Load the vectorizer
for key in dict_df_nacional69:
    df_key = dict_df_nacional69[key]
    if df_key.empty:
        print(f"The DataFrame for key '{key}' is empty. Skipping...")
        continue
    df_key[['Predicción Clave', 'Puntaje Confianza']] = df_key.apply(
        lambda row: pd.Series(classify_text(model, vectorizer, row['Descripción']), index=['Predicción Clave', 'Puntaje Confianza']), 
        axis=1
    )

# Esto es solo para mostrar
from IPython.display import display
for key in dict_df_nacional69:
    display(dict_df_nacional69[key])

# Listo :)

# --- GUARDAR
# Crea un archivo nuevo
wb = Workbook()
# O abre un archivo existente
#workbook_path = 'path_to_your_existing_workbook.xlsx'  # Replace with your file path
#wb = load_workbook(workbook_path)
default_sheet = wb.active
wb.remove(default_sheet)
sheet_name = 'TC Nacional 69' # Nombre planilla
ws = wb.create_sheet(title=sheet_name)
row_index = 1
write_df_dict(ws, dict_df_nacional69, 'TC Nacional 69', row_index)
# Save the workbook
wb.save(f"tc69.xlsx")

1.TOTAL OPERACIONES


ValueError: Length of values (5) does not match length of index (3)

## **TC INTERNACIONAL**

In [ ]:
import pdfplumber
from collections import defaultdict
import pandas as pd
pdf_path = r'C:\Users\czamora\Downloads\EstadoDeCuentaInternacional (2).pdf'
pdf_path = './documentos_1224/EstadoDeCuentaInternacional_24_dic24.pdf'
pdf_path = './documentos_0125/EstadoDeCuentaInternacional_24_ene25.pdf'

tables = []

with pdfplumber.open(pdf_path) as pdf:
    page_list = pdf.pages
    for page_number, page in enumerate(page_list):
        if page_number == 0:
            table = page.extract_tables()[-1][2:]
        elif page_number > 0:
            table = page.extract_tables()[0][2:]
        tables.append(table)

header_list = ['TOTAL DE PAGOS', 'TOTAL DE COMPRAS', 'COMISIONES, OTROS CARGOS Y ABONOS A LA CUENTA']
col_list = ['Número referencia', 'Fecha', 'Descripción', 'Ciudad', 'País', 'Monto', 'Monto US']

dict_head = {}
current_header = None
len_list = []
for table in tables:
    print()
    for row in table:
        len_cols = len(row)
        len_list.append(len_cols)
        #row = row[:-3]
        if row[0].startswith(tuple(header_list)):
            current_header = row[0].split(' US')[0]
            #print(current_header)
            dict_head[current_header] = defaultdict(list)
        else:
            for i, element in enumerate(row):
                # --- CORRECCION SEPT 2024 (Separar columnas monto)
                print(element.split('\n'), print(col_list[i]))
                # if col_list[i] == 'Monto':
                #     #print(element.split('\n'))
                #     dict_head[current_header][col_list[i]] += [e.split()[-1] for e in element.split('\n')]
                # else:
                # # ---
                dict_head[current_header][col_list[i]] += element.split('\n')
if len(set(len_list)) != 1:
    print('Existe un problema con la separación de las columnas, revisar. ')

string = 'TOTAL TARJETA XXXXXXXXXXXX8724'
dict_df_inter = {}
for header in header_list:
    df = pd.DataFrame()
    for i, col in enumerate(['Número referencia', 'Fecha', 'Descripción', 'Ciudad', 'Monto', 'Monto US']):
        #print(dict_head[header][col])
        #print(f'{i} es columna {nombre_col}')
        if header == 'TOTAL DE COMPRAS':
            #print(f'En modo {op}')
            if string in dict_head[header][col]:
                dict_head[header][col].remove(string)
            # if col in ['Descripción', 'Monto']:
            #     df[col] = dict_head[header][col][:-1]
            #else:
            df[col] = dict_head[header][col]
        else:
            df[col] = dict_head[header][col]
    dict_df_inter[header] = df

# for key in dict_df_inter:
#     print(dict_df_inter[key].to_string())

for key in dict_df_inter:
    df = dict_df_inter[key]
    df['Monto USD'] = df['Monto'].apply(lambda x: float(x.replace(',', '.')))
    df['Monto CLP'] = df['Monto USD'] * PRECIO_DOLAR
    df['Descripción'] = df['Descripción'].astype(str)
    df['Ciudad'] = df['Ciudad'].fillna('').astype(str)
    df['Descripción'] = df['Descripción'] + ' ' + df['Ciudad']
    df = df.drop(columns=['Monto', 'Ciudad'])
    dict_df_inter[key] = df

# APLICACION MODELO PREDICTIVO ---
# Cargar modelo
with open("./modelo_TCI.pkl", "rb") as model_file, open("vectorizer_TCI.pkl", "rb") as vectorizer_file:
    model = pickle.load(model_file)  # Load the model
    vectorizer = pickle.load(vectorizer_file)  # Load the vectorizer
for key in dict_df_inter:
    df_key = dict_df_inter[key]
    if df_key.empty:
        #print(f"The DataFrame for key '{key}' is empty. Skipping...")
        continue
    df_key[['Predicción Clave', 'Puntaje Confianza']] = df_key.apply(
        lambda row: pd.Series(classify_text(model, vectorizer, row['Descripción']), index=['Predicción Clave', 'Puntaje Confianza']), 
        axis=1
    )

from IPython.display import display
for key in dict_df_inter:
    display(dict_df_inter[key])

# # --- GUARDAR
# # Crea un archivo nuevo
# wb = Workbook()
# # O abre un archivo existente
# #workbook_path = 'path_to_your_existing_workbook.xlsx'  # Replace with your file path
# #wb = load_workbook(workbook_path)
# default_sheet = wb.active
# wb.remove(default_sheet)
# sheet_name = 'TC Internacional' # Nombre planilla
# ws = wb.create_sheet(title=sheet_name)
# row_index = 1
# write_df_dict(ws, dict_df_inter, 'TC Internacional', row_index)
# # Save the workbook
# wb.save(f"tcinter.xlsx")


Número referencia
['0801 24204295007000511278049', '0801 74143615008000058000936', '1301 74143615011000059751623', '1301 74143615012000063130706', '1301 74143615012000063133999', '1601 74143615016000061501177', '2001 74609055018000001203060', '2701 24011345027500030927549', '2901 74143615029000059556055', '0302 24692165032106496107062'] None
Fecha
['07/01/25', '07/01/25', '10/01/25', '11/01/25', '11/01/25', '15/01/25', '18/01/25', '27/01/25', '28/01/25', '01/02/25'] None
Descripción
['SiteMinder Hospital', 'FACEBK *SHSZYEQH62', 'FACEBK *XPBA5G8H62', 'FACEBK *4WPJ7GGH62', 'FACEBK *FST4KFYG62', 'FACEBK *F36TDGGH62', 'ILOVEPDF.COM', 'WWW.HOTELOGIX.COM', 'FACEBK *26CA2HGH62', 'GOOGLE *GSUITE_pasc', 'TOTAL TARJETA XXXXXXXXXXXX8724'] None
Ciudad
['029-2214444', 'DUBLIN', 'DUBLIN', 'DUBLIN', 'DUBLIN', 'DUBLIN', 'BARCELONA', 'WWW.', 'DUBLIN', 'cc@google.c'] None
País
['US 249', 'IE 17.716', 'IE 19.488', 'IE 1.092', 'IE 6.106', 'IE 21.437', 'ES 7', 'US 108', 'IE 23.581', 'US 64'] None
Monto
['

ValueError: Length of values (11) does not match length of index (10)

## **Transbank**

In [ ]:
import pandas as pd
# nombre_list = ['./ERR_02_25/download.xls', './ERR_02_25/download (1).xls']
# df_list = []
# header = 16
# for nombre_archivo in nombre_list:
#     df_list.append(pd.read_excel(nombre_archivo, header=header))
# df_stacked = pd.concat(df_list, axis=0)

header = 15
nombre_archivo = './ERR_04_25/Cartola_Movimientos_transbank_01 al 30 abr. 2025.xlsx'
df_stacked = pd.read_excel(nombre_archivo, header=header)
df_stacked = df_stacked.iloc[:, 1:]

print(df_stacked.to_string())

df_stacked = df_stacked.loc[:, ~df_stacked.columns.str.startswith('Unnamed:')]
#Safely split the first column into two parts
split_columns = df_stacked.iloc[:, 0].str.split(' ', n=1, expand=True)
#Assign the results, filling missing values with NaN
df_stacked['Fecha'] = split_columns[0]
df_stacked['Hora'] = split_columns[1] if split_columns.shape[1] > 1 else None
#Drop the original first column if no longer needed
df_stacked = df_stacked.drop(df_stacked.columns[0], axis=1)
#Split fecha hora

def safe_int(x):
    try:
        return int(float(x))  # Handles numbers like "12.0" or "  5"
    except (ValueError, TypeError):
        return ' '

# Crea columna Pesos
def calculate_pesos(row):
    if row['Tipo movimiento'] == 'Venta USD$':
        return row['Total'] * PRECIO_DOLAR
    else:
        return row['Total']
    
def get_moneda(row):
    if row['Tipo movimiento'] == 'Venta USD$':
        return 'Dolar'
    else:
        return 'Peso'

dict_names = {'Fecha' : 'Fecha Venta', 'Local' : 'Codigo comercio', 'Tipo Movimiento' : 'Tipo movimiento', 'Identificador' : 'Número tarjeta', 'Monto Afecto' : 'Total', 'N° Boleta' : 'Número'}
new_cols = ['Tipo venta', 'Monto', 'Propina', 'Vuelto', 'Tipo cuota', 'Monto cuota', 'Número cuotas', 'Tarjeta', 'Código empleado', 'Número boleta', 'Código autorización']

# Rename the columns according to the dictionary
df_stacked = df_stacked.rename(columns=dict_names)

# Select only the columns that are in the dict_names keys
df_stacked = df_stacked[list(dict_names.values())]

df_stacked['Pesos $'] = df_stacked.apply(calculate_pesos, axis=1)
df_stacked['Moneda'] = df_stacked.apply(get_moneda, axis=1)
#df_stacked['Número'] = df_stacked['Número'].apply(lambda x: int(x) if not np.isnan(x) else ' ')
#df_stacked['Número'] = df_stacked['Número'].apply(lambda x: int(x) if not pd.isna(x) else ' ')
df_stacked['Número'] = df_stacked['Número'].apply(safe_int)

# Add the new columns from new_cols as empty columns
for col in new_cols:
    df_stacked[col] = ""

df_transbank = df_stacked[['Número', 'Pesos $', 'Fecha Venta', 'Codigo comercio', 'Tipo venta', 'Tipo movimiento', 'Monto', 'Propina', 'Vuelto', 'Total', 'Tipo cuota', 'Monto cuota', 'Número cuotas', 'Moneda', 'Tarjeta', 'Número tarjeta', 'Código empleado', 'Número boleta', 'Código autorización']]
df_transbank['Codigo comercio'] = df_transbank['Codigo comercio'].apply(lambda x: str(x))
#df_transbank

# --- GUARDAR
# Crea un archivo nuevo
wb = openpyxl.Workbook()
# O abre un archivo existente
#workbook_path = 'path_to_your_existing_workbook.xlsx'  # Replace with your file path
#wb = load_workbook(workbook_path)
default_sheet = wb.active
wb.remove(default_sheet)
sheet_name = 'Transbank' # Nombre planilla
ws = wb.create_sheet(title=sheet_name)
row_index = 1
write_df(ws, df_transbank, 'Transbank', row_index)
# Save the workbook
wb.save(f"out_04_25/transbank.xlsx")

    Fecha Venta     Local                        Identificación Local Tipo Movimiento Tipo Tarjeta     Identificador Tipo Cuota  Monto Afecto  Monto Exento Código Autorización N° Cuotas  Monto Cuota Primer Abono   N° Boleta  Monto Vuelto
0    30/04/2025  42558647  HOTEL PASCUAL ANDINO GUSTAVO LE PAIGE 150       Venta USD$           VI  XXXXXXXXXXXX2303  Venta US$        589.95             0              798066       S/C            0   07/05/2025  0000000056             0
1    30/04/2025  42558647  HOTEL PASCUAL ANDINO GUSTAVO LE PAIGE 150       Venta USD$           VI  XXXXXXXXXXXX4367  Venta US$        831.05             0              078945       S/C            0   05/05/2025  0000000055             0
2    29/04/2025  42558647  HOTEL PASCUAL ANDINO GUSTAVO LE PAIGE 150       Venta USD$           MC  XXXXXXXXXXXX6854  Venta US$        176.99             0              975783       S/C            0   05/05/2025  0000000054             0
3    29/04/2025  42558647  HOTEL PASCUAL ANDINO 

In [54]:
import pandas as pd
from openpyxl.utils import get_column_letter

header = 16 # Row number in excel
name_file = './ERR_04_25/Cartola_Movimientos_transbank_01 al 30 abr. 2025.xlsx'
df_transbank = pd.read_excel(name_file, header=header - 1)
# split_columns = df_transbank.iloc[:, 0].astype(str).str.split(' ', n=1, expand=True)
# df_transbank['Fecha'] = split_columns[0]
# df_transbank['Hora'] = split_columns[1] if split_columns.shape[1] > 1 else None
# df_transbank = df_transbank.drop(df_transbank.columns[0], axis=1)

# Dict maps Excel headers → DataFrame headers
column_dict = {
    'NUMERO': 'N° Boleta',
    'FECHA': 'Fecha Venta',
    'CODIGO COMERCIO': 'Local',
    'TIPO MOVIMIENTO': 'Tipo Movimiento',
    'TOTAL': 'Monto Afecto',
    'NUMERO TARJETA': 'Identificador',
}

# Load the Excel template
template_path = './EERR_modelo.xlsx'
wb = load_workbook(template_path)
ws = wb["Transbank"]

# Read Excel headers from specified header row
header_row = 3
headers = [cell.value for cell in ws[header_row]]

# Map Excel headers to column letters (only for those we want to fill)
header_positions = {}
for idx, excel_header in enumerate(headers):
    if excel_header in column_dict:
        col_letter = get_column_letter(idx + 1)
        header_positions[excel_header] = col_letter

# Write values to the sheet
start_row = 4  # Row after the headers (assumed)
for i, row in df_transbank.iterrows():
    for excel_header, df_column in column_dict.items():
        if excel_header in header_positions and df_column in df_transbank.columns:
            col_letter = header_positions[excel_header]
            cell = f"{col_letter}{start_row + i}"
            ws[cell] = row[df_column]

# Save to output
output_path = "outputs/output_filled.xlsx"
wb.save(output_path)

## **Site**

In [9]:
df_site = pd.read_csv('./ERR_05_25/reservations_summary_report_site_may25.csv')
df_site.columns

Index(['Referencia de la reserva', 'Nombres de los huéspedes', 'Llegada',
       'Salida', 'Canal', 'Affiliated Channel', 'Referral', 'Habitación',
       'Fecha de reserva', 'Estado de la reserva', 'Ocupación', 'Precio total',
       'Estado de la conexión', 'Última conexión establecida', 'Guest title',
       'Guest name', 'Guest first name', 'Guest last name', 'Guest address',
       'Guest address2', 'Guest state', 'Guest postcode', 'Guest country',
       'Guest email', 'Guest phone', 'GDPR consent', 'GDPR consent date',
       'Promo code'],
      dtype='object')

In [11]:
from datetime import datetime
import pandas as pd
import openpyxl

#df_site = pd.read_csv('./ERR_03_25/reservations_summary_report_mar25.csv')
df_site = pd.read_csv('./ERR_05_25/reservations_summary_report_site_may25.csv')

def get_first_e(row, string=' '):
    return row.split(string)[0]
def get_last_e(row, string=' '):
    return row.split(string)[-1]
def convert_date(row):
    #return datetime.strptime(row, "%m/%d/%Y")
    return datetime.strptime(row, "%Y-%m-%d")

print(list(df_site.columns))

dict_names = {'Referencia de la reserva' : 'ID Reserva', 'Nombres de los huéspedes' : 'Huéspedes', 'Llegada' : 'Check-In', 'Salida' : 'Check-Out', 'Affiliated Channel' :'Canal afiliado', 'Estado de la reserva' : 'Estado', 'Ocupación' : 'Noches', 'Habitación' : 'Habitaciones', 'Precio total' : 'Total'}
df_site = df_site.rename(columns=dict_names)
#df_site['Noches'] = df_site['Noches'].apply(get_first_e, ' ')
df_site['Habitaciones'] = df_site['Habitaciones'].apply(get_first_e)
df_site['Fecha de reserva'] = df_site['Fecha de reserva'].apply(get_first_e)
df_site['Total'] = df_site['Total'].apply(get_last_e)
df_site['Total'] = df_site['Total'].astype(float)
df_site['Habitaciones'] = df_site['Habitaciones'].astype(int)
print(df_site['Check-In'])
df_site['Noches'] = df_site['Check-Out'].apply(convert_date) - df_site['Check-In'].apply(convert_date)
df_site['Noches'] = df_site['Noches'].dt.days

df_site['Index ocupación'] = df_site['Noches'] * df_site['Habitaciones']
df_site['Tarifa'] = df_site['Total']/df_site['Index ocupación']

df_site = df_site[['ID Reserva', 'Huéspedes', 'Check-In', 'Check-Out', 'Canal', 'Canal afiliado', 'Fecha de reserva', 'Estado', 'Total', 'Noches', 'Habitaciones', 'Tarifa', 'Index ocupación']]
df_site
#df_site = df_site[['Fecha de reserva'] + list(dict_names.values()) + ['Tarifa', 'Index ocupación']]

# --- GUARDAR
# Crea un archivo nuevo
wb = openpyxl.Workbook()
# O abre un archivo existente
#workbook_path = 'path_to_your_existing_workbook.xlsx'  # Replace with your file path
#wb = load_workbook(workbook_path)
default_sheet = wb.active
wb.remove(default_sheet)
sheet_name = 'Site' # Nombre planilla
ws = wb.create_sheet(title=sheet_name)
row_index = 1
write_df(ws, df_site, 'Site', row_index)
# Save the workbook
wb.save(f"./out_05_25/site.xlsx")

['Referencia de la reserva', 'Nombres de los huéspedes', 'Llegada', 'Salida', 'Canal', 'Affiliated Channel', 'Referral', 'Habitación', 'Fecha de reserva', 'Estado de la reserva', 'Ocupación', 'Precio total', 'Estado de la conexión', 'Última conexión establecida', 'Guest title', 'Guest name', 'Guest first name', 'Guest last name', 'Guest address', 'Guest address2', 'Guest state', 'Guest postcode', 'Guest country', 'Guest email', 'Guest phone', 'GDPR consent', 'GDPR consent date', 'Promo code']
0     2025-05-01
1     2025-05-02
2     2025-05-02
3     2025-05-03
4     2025-05-03
5     2025-05-03
6     2025-05-04
7     2025-05-04
8     2025-05-05
9     2025-05-05
10    2025-05-05
11    2025-05-06
12    2025-05-06
13    2025-05-07
14    2025-05-09
15    2025-05-09
16    2025-05-10
17    2025-05-11
18    2025-05-11
19    2025-05-11
20    2025-05-11
21    2025-05-12
22    2025-05-12
23    2025-05-12
24    2025-05-12
25    2025-05-13
26    2025-05-14
27    2025-05-16
28    2025-05-16
29    202

In [46]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter

# Load DataFrame
df_site = pd.read_csv('./ERR_04_25/reservations_summary_report_siteabr_25.csv')

# Dict maps Excel headers → DataFrame headers
column_dict = {
    'Reservation ID': 'Referencia de la reserva',
    'Guest': 'Nombres de los huéspedes',
    'Check In': 'Llegada',
    'Check Out': 'Salida',
    'Channel': 'Canal',
    'Date Created': 'Fecha de reserva',
    'Status': 'Estado de la reserva',
    'Total Amount': 'Precio total',
    'Habitaciones': 'Habitación'
}

# Load the Excel template
template_path = './EERR_modelo.xlsx'
wb = load_workbook(template_path)
ws = wb["S"]

# Read Excel headers from specified header row
header_row = 4
headers = [cell.value for cell in ws[header_row]]

# Map Excel headers to column letters (only for those we want to fill)
header_positions = {}
for idx, excel_header in enumerate(headers):
    if excel_header in column_dict:
        col_letter = get_column_letter(idx + 1)
        header_positions[excel_header] = col_letter

# Write values to the sheet
start_row = 5  # Row after the headers (assumed)
for i, row in df_site.iterrows():
    for excel_header, df_column in column_dict.items():
        if excel_header in header_positions and df_column in df_site.columns:
            col_letter = header_positions[excel_header]
            cell = f"{col_letter}{start_row + i}"
            ws[cell] = row[df_column]

# Save to output
output_path = "outputs/output_filled.xlsx"
wb.save(output_path)

## **Security**

In [ ]:
import pickle
from openpyxl import load_workbook
from text_classification import classify_text
from openpyxl.utils import get_column_letter

#nombre_archivo = './ERR_03_25/CartolaHistorica_security_mar25.xlsx'
nombre_archivo = './ERR_05_25/CartolaHistorica_security_abr25.xls.xlsx'
header = 13
df_sec = pd.read_excel(nombre_archivo, header=header, thousands='.')

# Initialize indices
end_index = None
# Iterate through the rows to find the indices of "FECHA" and "Resumen"
for i, row in df_sec.iterrows():
    if pd.isna(row[0]):
         end_index = i
         break  # Stop iteration once "Resumen" is found
df_sec = df_sec.iloc[: end_index]
for col in ['cargos ', 'abonos ', 'saldos ']:
    df_sec[col] = df_sec[col].apply(lambda x: int(float(x.replace(',', '.'))))
# Cambio de nombre de las columnas
dict_names = {'fecha ' : 'Fecha', 'descripción ' : 'Descripción', 'número de documentos ' : 'N° Documento', 'cargos ' : 'Cargo', 'abonos ' : 'Abono', 'saldos ' : 'Saldo'}
df_sec = df_sec.rename(columns=dict_names)

# Convert to datetime format
df_sec['Fecha'] = pd.to_datetime(df_sec['Fecha'])

# Format to the desired string format
df_sec['Fecha'] = df_sec['Fecha'].dt.strftime('%d/%m/%Y')

# APLICACION MODELO PREDICTIVO ---
# Cargar modelo
with open("./modelo_security.pkl", "rb") as model_file, open("./vectorizer_security.pkl", "rb") as vectorizer_file:
    model = pickle.load(model_file)  # Load the model
    vectorizer = pickle.load(vectorizer_file)  # Load the vectorizer
# Aplicar
df_sec[['Predicción Clave', 'Puntaje Confianza']] = df_sec.apply(
    lambda row: pd.Series(classify_text(model, vectorizer, row['Descripción'])),
    axis=1
)

print(df_sec.to_string())

# --- GUARDAR
# Crea un archivo nuevo
wb = openpyxl.Workbook()
# O abre un archivo existente
#workbook_path = 'path_to_your_existing_workbook.xlsx'  # Replace with your file path
#wb = load_workbook(workbook_path)
default_sheet = wb.active
wb.remove(default_sheet)
sheet_name = 'Security' # Nombre planilla
ws = wb.create_sheet(title=sheet_name)
row_index = 1
write_df(ws, df_sec, 'Security', row_index)
# Save the workbook
wb.save(f"./out_04_25/security.xlsx")

         Fecha                                             Descripción N° Documento    Cargo   Abono     Saldo Predicción Clave  Puntaje Confianza
0   04/01/2025                           ABONO REALIZADO POR TRANSBANK            0        0  638460  19882507   Transf Ingreso                1.0
1   04/02/2025                           ABONO REALIZADO POR TRANSBANK            0        0  893960  20776467   Transf Ingreso                1.0
2   04/07/2025                           ABONO REALIZADO POR TRANSBANK            0        0  197602  20974069   Transf Ingreso                1.0
3   04/08/2025                           ABONO REALIZADO POR TRANSBANK            0        0  408070  21382139   Transf Ingreso                1.0
4   04/11/2025                     COTIZACIONES PREVISIONALES-PREVIRED    688576667  2483548       0  18898591          Sueldos                0.9
5   15/04/2025                                       PAGO EN LINEA SII    690648659  3123305       0  15775286        

NameError: name 'openpyxl' is not defined

In [4]:
import pickle
from openpyxl import load_workbook
from text_classification import classify_text
import pandas as pd
from openpyxl.utils import get_column_letter

name_file = './ERR_04_25/CartolaHistorica_security_abr25.xls.xlsx'
header = 14
df_sec = pd.read_excel(name_file, header=header - 1, thousands='.')
end_index = None
# Itera hasta encontrar un espacio y para
for i, row in df_sec.iterrows():
    if pd.isna(row[0]):
         end_index = i
         break 
df_sec = df_sec.iloc[: end_index]

# APLICACION MODELO PREDICTIVO ---
# Cargar modelo
with open("./modelo_security.pkl", "rb") as model_file, open("./vectorizer_security.pkl", "rb") as vectorizer_file:
    model = pickle.load(model_file)  # Load the model
    vectorizer = pickle.load(vectorizer_file)  # Load the vectorizer
# Aplicar
df_sec[['Predicción Clave', 'Puntaje Confianza']] = df_sec.apply(
    lambda row: pd.Series(classify_text(model, vectorizer, row['descripción '])),
    axis=1
)

# Dict maps Excel headers → DataFrame headers
column_dict = {
    'Fecha ': 'fecha ',
    'Descripción ': 'descripción ',
    'Nº documento ': 'número de documentos ',
    ' Cargos ': 'cargos ',
    'Abonos ': 'abonos ',
    '$': 'saldos ',
    'tipo' : 'Predicción Clave'
}

# Load the Excel template
template_path = './EERR_modelo.xlsx'
wb = load_workbook(template_path)
ws = wb["Security"]

# Read Excel headers from specified header row
header_row = 1
headers = [cell.value for cell in ws[header_row]]

# Map Excel headers to column letters (only for those we want to fill)
header_positions = {}
for idx, excel_header in enumerate(headers):
    if excel_header in column_dict:
        col_letter = get_column_letter(idx + 1)
        header_positions[excel_header] = col_letter

# Write values to the sheet
start_row = header_row + 1 # Row after the headers (assumed)
for i, row in df_sec.iterrows():
    for excel_header, df_column in column_dict.items():
        if excel_header in header_positions and df_column in df_sec.columns:
            col_letter = header_positions[excel_header]
            cell = f"{col_letter}{start_row + i}"
            ws[cell] = row[df_column]

# Save to output
output_path = "outputs/output_filled.xlsx"
wb.save(output_path)

# Normalizar fechas ----
print(df_sec.to_string())

                 fecha                                             descripción  número de documentos      cargos     abonos       saldos  Predicción Clave  Puntaje Confianza
0   2025-01-04 00:00:00                           ABONO REALIZADO POR TRANSBANK                     0        0,00  638460,00  19882507,00   Transf Ingreso                1.0
1   2025-02-04 00:00:00                           ABONO REALIZADO POR TRANSBANK                     0        0,00  893960,00  20776467,00   Transf Ingreso                1.0
2   2025-07-04 00:00:00                           ABONO REALIZADO POR TRANSBANK                     0        0,00  197602,00  20974069,00   Transf Ingreso                1.0
3   2025-08-04 00:00:00                           ABONO REALIZADO POR TRANSBANK                     0        0,00  408070,00  21382139,00   Transf Ingreso                1.0
4   2025-11-04 00:00:00                     COTIZACIONES PREVISIONALES-PREVIRED             688576667  2483548,00       0,00  1889

## **Formato**

In [58]:
from openpyxl.styles import PatternFill, Font, Alignment

# Define the styles
header_fill = PatternFill(start_color="C5D9F1", end_color="C5D9F1", fill_type="solid")
header_font = Font(bold=True, color="000000", size=12)

title_font = Font(bold=True, color="EEEEEE", size=12)
title_fill = PatternFill(start_color="999999", end_color="999999", fill_type="solid")

mid_title_fill = PatternFill(start_color="EEEEEE", end_color="EEEEEE", fill_type="solid")
mid_title_font = Font(bold=True, color="000000", size=12)

def write_df(ws, df, title, start_row):
    global row_index
    # Write the single DataFrame with its title
    ws.merge_cells(start_row=start_row, start_column=1, end_row=start_row, end_column=len(df.columns))
    title_cell = ws.cell(row=start_row, column=1)
    title_cell.value = title
    title_cell.font = title_font
    title_cell.fill = title_fill
    title_cell.alignment = Alignment(horizontal="center", vertical="center")

    row_index += 1  # Move to the next row

    # Write the DataFrame header
    for col_index, col_name in enumerate(df.columns, start=1):
        cell = ws.cell(row=row_index, column=col_index)
        cell.value = col_name
        cell.font = header_font
        cell.fill = header_fill

    row_index += 1  # Move to the next row

    # Write the DataFrame content
    for r in df.itertuples(index=False):
        for col_index, value in enumerate(r, start=1):
            cell = ws.cell(row=row_index, column=col_index)
            cell.value = value
            # Apply number formatting for columns that should have thousand separators
            # Exclude the "N° de documento" column
            column_header = df.columns[col_index - 1]  # Adjust index since col_index is 1-based
            if isinstance(value, (int, float)) and column_header != 'N° Documento':
                cell.number_format = '#,##0'
        row_index += 1  # Move to the next row

def write_df_dict(ws, df_dict, title, start_row):
    global row_index
    # Write the title for the dictionary
    ws.merge_cells(start_row=start_row, start_column=1, end_row=start_row, end_column=len(df_dict[list(df_dict.keys())[0]].columns))
    title_cell = ws.cell(row=start_row, column=1)
    title_cell.value = title
    title_cell.font = title_font
    title_cell.fill = title_fill
    title_cell.alignment = Alignment(horizontal="center", vertical="center")
    
    row_index = start_row + 1  # Move to the next row

    # Write the shared header
    col_index = 1
    for col_name in df_dict[list(df_dict.keys())[0]].columns:
        cell = ws.cell(row=row_index, column=col_index)
        cell.value = col_name
        cell.font = header_font
        cell.fill = header_fill
        col_index += 1
        
    row_index += 1  # Move to the next row

    # Write each DataFrame in the dictionary, with its title
    for key, df in df_dict.items():
        # Write the key as a title
        ws.merge_cells(start_row=row_index, start_column=1, end_row=row_index, end_column=len(df.columns))
        title_cell = ws.cell(row=row_index, column=1)
        title_cell.value = key
        title_cell.font = mid_title_font
        title_cell.fill = mid_title_fill
        title_cell.alignment = Alignment(horizontal="center", vertical="center")
        
        row_index += 1  # Move to the next row
        
        # Write the DataFrame content
        for r in df.itertuples(index=False):
            for col_index, value in enumerate(r, start=1):
                cell = ws.cell(row=row_index, column=col_index)
                cell.value = value
                
                # Apply number formatting for columns that should have thousand separators
                # Exclude the "N° de documento" column
                column_header = df.columns[col_index - 1]  # Adjust index since col_index is 1-based
                if isinstance(value, (int, float)) and column_header != 'N° Documento':
                    cell.number_format = '#,##0'
            row_index += 1  # Move to the next row
        
        row_index += 1  # Add space between DataFrames

In [54]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import NamedStyle
from openpyxl.utils import get_column_letter

# Example data (replace with your actual data)

# Crea un archivo nuevo
wb = Workbook()

# O abre un archivo existente
#workbook_path = 'path_to_your_existing_workbook.xlsx'  # Replace with your file path
#wb = load_workbook(workbook_path)

default_sheet = wb.active
wb.remove(default_sheet)

# BCIFondRend
sheet_name = 'BCIFondRend'
ws = wb.create_sheet(title=sheet_name)
row_index = 1
write_df_dict(ws, dict_df_cta18, 'BCI Cuenta 18', row_index)

# BCI
sheet_name = "BCI"
ws = wb.create_sheet(title=sheet_name)

row_index = 1
# Escribe el primer DataFrame
write_df(ws, df_cta85, 'BCI Cuenta 85', row_index)
row_index += 1
write_df(ws, df_cta_cte, 'BCI Comercio Exterior', row_index)
row_index += 1
# Escribe DataFrames en el dict tc nacional24
write_df_dict(ws, dict_df_nacional24, 'TC Nacional 24', row_index)
row_index += 1
# Escribe DataFrames en el dict tc nacional69
write_df_dict(ws, dict_df_nacional69, 'TC Nacional 69', row_index)
row_index += 1
# Escribe DataFrames en el dict tc nacional
write_df_dict(ws, dict_df_inter, 'TC Internacional', row_index)
row_index += 1

# Transbank
sheet_name = 'Transbank'
ws = wb.create_sheet(title=sheet_name)
row_index = 1
write_df(ws, df_transbank, 'Transbank', row_index)

# Site
sheet_name = 'S'
ws = wb.create_sheet(title=sheet_name)
row_index = 1
write_df(ws, df_site, 'Reservas', row_index)

# Security
sheet_name = 'Security'
ws = wb.create_sheet(title=sheet_name)
row_index = 1
write_df(ws, df_sec, 'Banco Security', row_index)

# Save the workbook
fecha = 'prueba'
wb.save(f"ERR_{fecha}.xlsx")

NameError: name 'df_transbank' is not defined

In [ ]:
from openpyxl import load_workbook
from openpyxl.utils import range_boundaries, get_column_letter

def move_named_range_block(filepath, named_range_name, offset_rows, sheet_name=None, save_as=None):
    wb = load_workbook(filepath)
    defined_name = wb.defined_names.get(named_range_name)
    if defined_name is None:
        raise ValueError(f"Named range '{named_range_name}' not found.")

    destinations = list(defined_name.destinations)
    if not destinations:
        raise ValueError(f"Named range '{named_range_name}' has no destinations.")

    for sheet_title, coord in destinations:
        if sheet_name and sheet_title != sheet_name:
            continue

        ws = wb[sheet_title]
        min_col, min_row, max_col, max_row = range_boundaries(coord)

        ws.move_range(
            f"{get_column_letter(min_col)}{min_row}:{get_column_letter(max_col)}{max_row}",
            rows=offset_rows, cols=0
        )

        new_range = (
            f"'{sheet_title}'!${get_column_letter(min_col)}${min_row + offset_rows}:"
            f"${get_column_letter(max_col)}${max_row + offset_rows}"
        )
        defined_name.attr_text = new_range
        break

    wb.save(save_as or filepath)

# Run test:
move_named_range_block(
    filepath="test.xlsx",
    named_range_name="Resumen_BCI",
    offset_rows=5,
    sheet_name="Sheet1",
    save_as="test_moved.xlsx"
)

In [24]:
from openpyxl import load_workbook
from openpyxl.utils import range_boundaries, get_column_letter

def move_named_range_block(filepath, named_range_name, offset_rows, sheet_name=None, save_as=None):
    wb = load_workbook(filepath)
    defined_name = wb.defined_names.get(named_range_name)
    if defined_name is None:
        raise ValueError(f"Named range '{named_range_name}' not found.")

    destinations = list(defined_name.destinations)
    if not destinations:
        raise ValueError(f"Named range '{named_range_name}' has no destinations.")

    for sheet_title, coord in destinations:
        if sheet_name and sheet_title != sheet_name:
            continue

        ws = wb[sheet_title]
        min_col, min_row, max_col, max_row = range_boundaries(coord)

        ws.move_range(
            f"{get_column_letter(min_col)}{min_row}:{get_column_letter(max_col)}{max_row}",
            rows=offset_rows, cols=0
        )

        new_range = (
            f"'{sheet_title}'!${get_column_letter(min_col)}${min_row + offset_rows}:"
            f"${get_column_letter(max_col)}${max_row + offset_rows}"
        )
        defined_name.attr_text = new_range
        break

    wb.save(save_as or filepath)

# Run test:
move_named_range_block(
    filepath="Book 4.xlsx",
    named_range_name="Resumen_BCI",
    offset_rows=5,
    sheet_name="Sheet1",
    save_as="Book 4 out.xlsx"
)

In [ ]:
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from copy import copy

def move_named_block(wb_path, sheet_name, start_row, end_row, target_start_row):
    wb = load_workbook(wb_path)
    ws = wb[sheet_name]
    
    offset = target_start_row - start_row
    n_rows = end_row - start_row + 1

    # Insert rows in destination so formulas can shift
    ws.insert_rows(target_start_row, amount=n_rows)

    for row in range(start_row, end_row + 1):
        for col in range(1, ws.max_column + 1):
            src_cell = ws.cell(row=row, column=col)
            dst_cell = ws.cell(row=row + offset, column=col)

            if src_cell.has_style:
                dst_cell._style = copy(src_cell._style)
            dst_cell.value = src_cell.value  # includes formulas
            dst_cell.number_format = src_cell.number_format

    wb.save("output_shifted.xlsx")


In [25]:
from openpyxl import load_workbook
from openpyxl.utils import range_boundaries
from copy import copy

def copy_named_range_to(wb_path, sheet_name, named_range, target_start_row):
    wb = load_workbook(wb_path)
    ws = wb[sheet_name]

    # Get named range definition
    named_range_obj = wb.defined_names[named_range]
    dests = list(named_range_obj.destinations)  # (sheetname, range)
    assert dests, f"Named range '{named_range}' not found."

    for sheet_name_ref, cell_range in dests:
        if sheet_name_ref != sheet_name:
            raise ValueError(f"Named range '{named_range}' is not on sheet '{sheet_name}'.")

        # Get start and end cell of named range
        min_col, min_row, max_col, max_row = range_boundaries(cell_range)
        n_rows = max_row - min_row + 1
        n_cols = max_col - min_col + 1

        # Insert empty rows at target location
        ws.insert_rows(target_start_row, amount=n_rows)

        for row_offset in range(n_rows):
            for col_offset in range(n_cols):
                src_cell = ws.cell(row=min_row + row_offset, column=min_col + col_offset)
                dst_cell = ws.cell(row=target_start_row + row_offset, column=min_col + col_offset)

                # Copy value or formula
                dst_cell.value = src_cell.value
                dst_cell.number_format = src_cell.number_format

                # Copy styles
                if src_cell.has_style:
                    dst_cell._style = copy(src_cell._style)

    # Save result
    wb.save("output_shifted_from_named_range.xlsx")

# 🔧 Usage Example
copy_named_range_to(
    wb_path="Book 4.xlsx",  # Replace with your file
    sheet_name="Sheet1",             # Sheet where the named range is defined
    named_range="Resumen_BCI",       # Name defined in Excel's Name Manager
    target_start_row=50              # New row where you want to insert the copied block
)